# League of Legends Data Analysis
### My self-taught python journey

I've always been curious to drill deeper into the details of League of Legends performance than available through the standard platforms. As I teach myself Python, I want to dive deeper into what makes a Solo-queue winner: how do top performers compare the the rest of us in terms of damage share, consistency, and macro play?

## Step 0: Import Necessary Modules

In [1]:
key = 'RGAPI-79b3b64d-ec83-4f23-96d6-b6b91a1a2aab'

In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import requests
import json
import time
from datetime import datetime

In [3]:
url_Core = 'https://na1.api.riotgames.com'
url_Summ_By_Name = '/lol/summoner/v4/summoners/by-name/'
url_Match_History = '/lol/match/v4/matchlists/by-account/'
url_Match_Stats = '/lol/match/v4/matches/'
url_Match_Timeline = '/lol/match/v4/timelines/by-match/'

In [4]:
lookup_name = 'jigglemyjag'

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Step 1: Import Necessary Data

In [6]:
def summRequest(name):
    ## input is a single string for summoner name
    ## output is json file
    url = f'{url_Core}{url_Summ_By_Name}{name}?api_key={key}'
    response = requests.get(url).json()
    #dataframe = json_normalize(response)
    return response

#### Explore the Match History Options 

In [7]:
json_normalize(summRequest(lookup_name))

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,SBBIrs6Aue0Nxu-uCkGafeHvzh_Pf25okIMwQAro-ddkB4w,D6vSjql1vG4RsEXLbzFueyKjizX8tMm6HWfUFFpKand1kg,O5cnL8OBx44QpOC01b-5K36rxKW0jZkSJdK5GJBr6UBATv...,JiggleMyJag,4403,1578119924000,84


Note that we <b>can improve our summRequest function by addint startIndex and endIndex functionality

In [8]:
def getGameIdLst(name, champion=None, queue=None, lane=None):
    ## Mandatory input is summoner name string
    ## Optional inputs filter for champion id and queue type
    ## output is list of gameIds that fulfill the criteria. riot caps list at 100 max
    paramkey = {'champion': champion, 'queue': queue, 'api_key': key}
    acct_ID = summRequest(name)['accountId']
    url = f'{url_Core}{url_Match_History}{acct_ID}'
    response = requests.get(url, params=paramkey).json()
    lst = [match['gameId'] for match in response['matches'] if (lane == None or match['lane'].lower()==lane.lower())]
    if (len(lst)) == 0:
        print("No games meet these search results")
    return lst

#gameIdLst = getGameIdLst('JiggleMyJag', queue=400)

#gameIdLst[:10]

In [9]:
def matchStats(matchID):
    ## Input: int gameId
    ## Output: dictionary w/individual match stats
    url = f'{url_Core}{url_Match_Stats}{matchID}?api_key={key}'
    response = requests.get(url).json()
    return response

# Comparing List Comprehensions vs for loops
### When performing operations on large datasets, efficiency is key

In [10]:
def matchHStatsDataframe(name, champion=None, queue=None, lane=None):
    ## returns an unpacked match history with following headers:
    ## gameId, platformId, gameCreation, gameDuration, queueId, mapId, seasonId, gameVersion, gameMode, gameType \
    ## teams, participants, participantIdentities
    lst = [matchStats(game) for game in getGameIdLst(name, champion, queue, lane)[:98]]
    df = pd.DataFrame.from_records(lst)
    return(df)

In [11]:
def matchHStatsDataframe_RateLimit(name, champion=None, queue=None, lane=None):
    ## returns an unpacked match history with following headers:
    ## gameId, platformId, gameCreation, gameDuration, queueId, mapId, seasonId, gameVersion, gameMode, gameType \
    ## teams, participants, participantIdentities
    lst = []
    for i in getGameIdLst(name, champion, queue, lane)[:98]:
        #time.sleep(2) 
        #Pass the rate limiter requirement
        #Can make this step a list comprehension if this step becomes necessary
        lst.append(matchStats(i))
    df = pd.DataFrame.from_records(lst)
    return(df)

In [12]:
## we cannot use timeit because the loop creates exceptional overhead, so we will instead compare time 
time.sleep(125)       
start = datetime.now()
match_hist = matchHStatsDataframe(lookup_name)
print (f'List comprehension function takes {datetime.now()-start} seconds' )

time.sleep(125)       
start = datetime.now()
y = matchHStatsDataframe_RateLimit(name=lookup_name)
print (f'Rate Limit function takes {datetime.now()-start} seconds' )

List comprehension function takes 0:01:01.841901 seconds
Rate Limit function takes 0:01:00.508333 seconds


After running the above code multiple times, <strong>the difference in timing between the two functions is negligible</strong>.

We will use the list comprehension, as it is more pythonic.

# Unpacking Dictionaries within a Dataframe
### This was a very annoying process for a novice. The first listed solution I came up with on my own, the second was developed after checking multiple stackoverflow sollutions

#### Step 1: Data Exploration
Let's first explore the dataframe so you know what I'm talking about. While the first columns may appear standard, the final three contain nested lists of dictionaries - how annoying! Obviously we can make separate dataframes with these contents, but we would then lose our index variable (gameId)

In [13]:
match_hist.head(2)

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teams,participants,participantIdentities
0,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."
1,3258267850,NA1,1578542251011,2337,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T...","[{'participantId': 1, 'teamId': 100, 'champion...","[{'participantId': 1, 'player': {'platformId':..."


#### Step 2: Unpacking

##### 2.1 Attempt One

Here is my first attempt at tackling this problem. Coming from an SQL background, I resolved to simply make a bajillion tables and join them together on similar keys (presumably gameId and participantId). 

It's messy, but it (technically) gets the job done

In [14]:
def unpackNestedObjs(df, attribute):
    temp = df
    lst = []
    count = 0
    if attribute in ['participants', 'participantIdentities', 'teams']:
        for key, row in temp.iterrows():
            count += 1
            try:
                for value in row[attribute]:
                    value['gameId'] = row.gameId #key # gameId
                    lst.append(value)
            except:
                print(f'error at game number {count}')
    else:
        for key, row in temp.iterrows():
            if 'participantId' not in row[attribute].keys():
                row[attribute]['participantId'] = row.participantId
            row[attribute]['gameId'] = key
            lst.append(row[attribute])
    response = pd.DataFrame.from_records(lst, index='gameId')                   #indexing off of gameId messes up calculation. find out why.
    #response.columns = ['participantId' if x == 'summonerId' else x for x in response.columns]
    return response

In [15]:
unpackNestedObjs(match_hist , 'participants')[:20:6]

,participantId,teamId,championId,spell1Id,spell2Id,stats,timeline
gameId,,,,,,,
3258355800,1,100,81,4,7,"{'participantId': 1, 'win': False, 'item0': 10...","{'participantId': 1, 'creepsPerMinDeltas': {'1..."
3258355800,7,200,143,4,14,"{'participantId': 7, 'win': True, 'item0': 302...","{'participantId': 7, 'creepsPerMinDeltas': {'1..."
3258267850,3,100,59,11,4,"{'participantId': 3, 'win': True, 'item0': 140...","{'participantId': 3, 'creepsPerMinDeltas': {'1..."
3258267850,9,200,92,4,12,"{'participantId': 9, 'win': False, 'item0': 30...","{'participantId': 9, 'creepsPerMinDeltas': {'1..."


You'll note that that none the data from the original Match History Dateframe is captured in this new dataframe barring the gameId variable, and that was only possible through adding multiple lines of code to add it conditionally. There must be a better way...

###### Attempt 2

Adding a multiindex with gameId preserves the original positional index and gameId when performing more granular analysis. In the example below, gameId becomes the left index, while the original numeric index becomes the right. 

In [16]:
idx = match_hist.set_index('gameId').participants.apply(pd.Series).stack().index
idx[:5]

MultiIndex([(3258355800, 0),
            (3258355800, 1),
            (3258355800, 2),
            (3258355800, 3),
            (3258355800, 4)],
           names=['gameId', None])

##### 2.1 hard-coded variables
So long as Riot keeps their data in a stable format, this should work. It does look a little messy, but it works well

In [17]:
def unpackNestedObjs2(df):
    #unpack participants   
    idx = df.set_index('gameId')['participants'].apply(pd.Series).stack().index 
    participants = pd.DataFrame(df['participants'].apply(pd.Series).stack().values.tolist())
    frame = df.merge(participants, on=['gameId'], how='left')
    #unpack participantIdentities
    idx = df.set_index('gameId')['participantIdentities'].apply(pd.Series).stack().index
    ids = pd.DataFrame(df.participantIdentities.apply(pd.Series).stack().values.tolist(), index=idx).reset_index().drop('level_1', 1)
    frame = frame.merge(ids, on = ['gameId', 'participantId'], how ='left')
    #unpack teams
    idx = df.set_index('gameId')['teams'].apply(pd.Series).stack().index
    teams = pd.DataFrame(df.teams.apply(pd.Series).stack().values.tolist(), index=idx).reset_index().drop('level_1', 1)
    frame = frame.merge(teams, on = ['gameId', 'teamId'], how ='left')

    frame.drop(columns=['participants', 'participantIdentities', 'teams'], inplace=True)

    for i in ['stats', 'timeline', 'player']:
        frame = frame.drop(i, 1).assign(**pd.DataFrame.from_records(frame[i].dropna().tolist()))
        
    frame.drop(columns = ['combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 
    'playerScore0', 'playerScore1', 'playerScore2', 'playerScore3','playerScore4', 
    'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9'], inplace=True)
    
    return frame

Let's make sure the output looks right: Yup, it does. <p> 
    
After scanning the dataframe, there are some columns that look like they're not used: <em> combatPlayerScore	objectivePlayerScore	totalPlayerScore	totalScoreRank	playerScore0	playerScore1	playerScore2	playerScore3	playerScore4	playerScore5	playerScore6	playerScore7	playerScore8	playerScore9. </em> <p>
    
I've gone ahead and added a line to remove the in the code below.

In [18]:
unpackNestedObjs2(match_hist)[:15:3]

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,participantId,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,damageTakenPerMinDeltas,role,lane,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenDiffPerMinDeltas,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,1,100,81,4,7,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",1055,1053,3153,3158,3078,0,3363,3,6,14,0,1,0,519,0,0,0,0,0,93553,12929,78269,2353,0,20195,5513,14659,22,4339,3,8527,5093,1052,14,3,21912,7283,13850,778,9575,9533,1,143,2,2.0,0.0,24,13,0,0,9.0,1.0,False,False,False,False,False,False,8005,1072,615,457,9111,435,340,0,9104,12,30,0,8014,305,0,0,8139,847,0,0,8135,1682,5,0,8000,8100,5005,5008,5003,"{'10-20': 5.8, '0-10': 4.7, '20-30': 3.8}","{'10-20': 382, '0-10': 251.10000000000002, '20...","{'10-20': 320.5, '0-10': 238.39999999999998, '...","{'10-20': 547, '0-10': 434, '20-30': 1036.9}",SOLO,BOTTOM,NaN,NaN,NaN,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,YouAreM1ne,-s4tIm93GjUqnWwgHUc9UWIrG_oTtZX-7Jyg-8Z9w-3iAziS,NA1,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,/v1/stats/player_history/NA1/2136899431172352,3898
3,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,4,100,39,12,4,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",3078,3053,1043,3047,1057,2033,3340,2,7,1,0,1,0,743,0,0,0,0,0,156485,11949,123495,21040,0,10422,3269,7023,130,5389,1,17332,3827,3827,11,12,24141,4533,18572,1036,11078,10328,0,233,0,0.0,0.0,63,15,1,0,7.0,0.0,False,False,False,False,False,False,8010,559,0,0,9111,212,60,0,9104,20,50,0,8014,196,0,0,8352,186,1168,716,8345,3,0,0,8000,8300,5005,5008,5002,"{'10-20': 9.5, '0-10': 6.699999999999999, '20-...","{'10-20': 503.40000000000003, '0-10': 404, '20...","{'10-20': 328.1, '0-10': 235.6, '20-30': 489.1}","{'10-20': 383.5, '0-10': 514.9, '20-30': 1515....",SOLO,TOP,NaN,NaN,NaN,nLNOcOE0ArNn_jTPWKmt6jCjflvROyftOnlat8AmUuVvqw,Ko1 M,OcahTpHPPpNf_qnGE_TtO90Q0HNaOBZzfJBq_Q-R3vzdkP8,NA1,nLNOcOE0ArNn_jTPWKmt6jCjflvROyftOnlat8AmUuVvqw,/v1/stats/player_history/NA1/46346263,4022
6,3258355800,NA,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,7,200,143,4,14,True,True,True,True,True,True,True,11,0,1,4,0,1,0,"[{'championId': 122, 'pickTurn': 6}, {'champio...",3020,3853,3165,2424,3136,2055,3364,2,9,9,0,1,0,364,

##### 2.2 Loops! 
Reformatting this code into a loopable format will be helpful if I ever need to use it again for another dataframe. Let's give it a shot:

In [19]:
def unpackNestedObjs_Loop(df):
    frame = df
    cols = ['participants', 'participantIdentities', 'teams']
    indexes = [['gameId'], ['gameId', 'participantId'], ['gameId', 'teamId']]
    for col, index in zip(cols, indexes):
        idx = df.set_index('gameId')[col].apply(pd.Series).stack().index
        if (len(index) == 1):
            temp = pd.DataFrame(df[col].apply(pd.Series).stack().values.tolist())
        else:
            temp = pd.DataFrame(df[col].apply(pd.Series).stack().values.tolist(), index = idx).reset_index().drop('level_1', 1)
        frame = frame.merge(temp, on = index, how = 'left')
        
    frame.drop(columns=cols, inplace=True)    
    for i in ['stats', 'timeline', 'player']:
        frame = frame.drop(i, 1).assign(**pd.DataFrame.from_records(frame[i].dropna().tolist()))
    
    frame.drop(columns = ['combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 
    'playerScore0', 'playerScore1', 'playerScore2', 'playerScore3','playerScore4', 
    'playerScore5', 'playerScore6', 'playerScore7', 'playerScore8', 'playerScore9'], inplace=True)
               
    return frame

##### A question for those more familiar with Python: why do I need to convert values.tolist() to make the above function work? It seems to construct single dataframes just fine in isolation, but once I put it the combined function it gives me a key error...

I actually do not know 

Let's quickly confirm that both methods return the same output:

In [20]:
unpackNestedObjs_Loop(match_hist).equals(unpackNestedObjs2(match_hist))

True

Now that we have two working methods, let's make sure there aren't any performance issues:

In [21]:
%timeit unpackNestedObjs2(match_hist)
%timeit unpackNestedObjs_Loop(match_hist)

217 ms ± 4.64 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
216 ms ± 2.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<b> Does the loop work? </b> Yes. <p>
<b> Is it faster? </b> A bit, yes. <p>
I'll be moving forward with the loop method as it's easier to follow. 

In [22]:
master = unpackNestedObjs_Loop(match_hist)
master.head(2)

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,participantId,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,damageTakenPerMinDeltas,role,lane,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenDiffPerMinDeltas,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,1,100,81,4,7,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",1055,1053,3153,3158,3078,0,3363,3,6,14,0,1,0,519,0,0,0,0,0,93553,12929,78269,2353,0,20195,5513,14659,22,4339,3,8527,5093,1052,14,3,21912,7283,13850,778,9575,9533,1,143,2,2.0,0.0,24,13,0,0,9.0,1.0,False,False,False,False,False,False,8005,1072,615,457,9111,435,340,0,9104,12,30,0,8014,305,0,0,8139,847,0,0,8135,1682,5,0,8000,8100,5005,5008,5003,"{'10-20': 5.8, '0-10': 4.7, '20-30': 3.8}","{'10-20': 382, '0-10': 251.10000000000002, '20...","{'10-20': 320.5, '0-10': 238.39999999999998, '...","{'10-20': 547, '0-10': 434, '20-30': 1036.9}",SOLO,BOTTOM,NaN,NaN,NaN,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,YouAreM1ne,-s4tIm93GjUqnWwgHUc9UWIrG_oTtZX-7Jyg-8Z9w-3iAziS,NA1,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,/v1/stats/player_history/NA1/2136899431172352,3898
1,3258355800,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,2,100,267,4,14,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",3107,3853,3504,1052,3117,0,3364,2,4,14,2,1,1,1216,0,0,0,0,0,17577,12133,3317,2126,0,10598,7453,1146,1999,9732,5,8164,604,287,59,30,19826,7418,11923,484,7163,6610,0,6,4,4.0,0.0,168,12,3,0,27.0,4.0,False,False,False,False,False,False,8214,768,635,0,8226,250,407,0,8234,12213,0,0,8237,484,0,0,8345,3,0,0,8347,0,0,0,8200,8300,5008,5008,5003,"{'10-20': 0.4, '0-10': 0, '20-30': 0.2}","{'10-20': 305.9, '0-10': 294.5, '20-30': 332.7}","{'10-20': 240, '0-10': 213.5, '20-30': 199.6}","{'10-20': 489.3, '0-10': 380.6, '20-30': 1071.3}",NONE,JUNGLE,"{'10-20': -4.6, '0-10': -4.55, '20-30': -2.9}","{'10-20': -252.59999999999997, '0-10': -153.70...","{'10-20': -133.44999999999993, '0-10': -69.349...",GCHrpTgHUKbSY0ZorVd8SsTGQYlGDZSie1mz-XH5-ua9jQ,DJfighter,s3PUgsZ49uZbXUaJffMFJNRktGmFDxv1IgBRFKKRVfwZXng,NA1,GCHrpTgHUKbSY0ZorVd8SsTGQYlGDZSie1mz-XH5-ua9jQ,/v1/stats/player_history/NA1/41939982,3178


It would certainly look nicer if <strong> both </strong> of our index columns werein the front, wouldn't it? </p>

I'm sure there's way to to this that doesn't involve temporary reindexing or making a reordered column list with every individual column, but considering this master data frame has so many columns, this is the fastest way I could think of: 

In [23]:
df = master
df.set_index(['gameId', 'summonerName', 'participantId'], inplace=True)
df.reset_index(inplace=True)
df.head(2)

,gameId,summonerName,participantId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,teamId,championId,spell1Id,spell2Id,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,largestKillingSpree,largestMultiKill,killingSprees,longestTimeSpentLiving,doubleKills,tripleKills,quadraKills,pentaKills,unrealKills,totalDamageDealt,magicDamageDealt,physicalDamageDealt,trueDamageDealt,largestCriticalStrike,totalDamageDealtToChampions,magicDamageDealtToChampions,physicalDamageDealtToChampions,trueDamageDealtToChampions,totalHeal,totalUnitsHealed,damageSelfMitigated,damageDealtToObjectives,damageDealtToTurrets,visionScore,timeCCingOthers,totalDamageTaken,magicalDamageTaken,physicalDamageTaken,trueDamageTaken,goldEarned,goldSpent,turretKills,totalMinionsKilled,neutralMinionsKilled,neutralMinionsKilledTeamJungle,neutralMinionsKilledEnemyJungle,totalTimeCrowdControlDealt,champLevel,visionWardsBoughtInGame,sightWardsBoughtInGame,wardsPlaced,wardsKilled,firstBloodKill,firstBloodAssist,firstTowerKill,firstTowerAssist,firstInhibitorKill,firstInhibitorAssist,perk0,perk0Var1,perk0Var2,perk0Var3,perk1,perk1Var1,perk1Var2,perk1Var3,perk2,perk2Var1,perk2Var2,perk2Var3,perk3,perk3Var1,perk3Var2,perk3Var3,perk4,perk4Var1,perk4Var2,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2,creepsPerMinDeltas,xpPerMinDeltas,goldPerMinDeltas,damageTakenPerMinDeltas,role,lane,csDiffPerMinDeltas,xpDiffPerMinDeltas,damageTakenDiffPerMinDeltas,accountId,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,3258355800,YouAreM1ne,1,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,100,81,4,7,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",1055,1053,3153,3158,3078,0,3363,3,6,14,0,1,0,519,0,0,0,0,0,93553,12929,78269,2353,0,20195,5513,14659,22,4339,3,8527,5093,1052,14,3,21912,7283,13850,778,9575,9533,1,143,2,2.0,0.0,24,13,0,0,9.0,1.0,False,False,False,False,False,False,8005,1072,615,457,9111,435,340,0,9104,12,30,0,8014,305,0,0,8139,847,0,0,8135,1682,5,0,8000,8100,5005,5008,5003,"{'10-20': 5.8, '0-10': 4.7, '20-30': 3.8}","{'10-20': 382, '0-10': 251.10000000000002, '20...","{'10-20': 320.5, '0-10': 238.39999999999998, '...","{'10-20': 547, '0-10': 434, '20-30': 1036.9}",SOLO,BOTTOM,NaN,NaN,NaN,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,-s4tIm93GjUqnWwgHUc9UWIrG_oTtZX-7Jyg-8Z9w-3iAziS,NA1,W4hZMhf6RMwX4LEaRuFUDfm9Mtzqjhf7QNlOw-Hs2xEwGg...,/v1/stats/player_history/NA1/2136899431172352,3898
1,3258355800,DJfighter,2,NA1,1578545344179,1825,440,11,13,10.1.303.9385,CLASSIC,MATCHED_GAME,100,267,4,14,False,False,False,False,False,False,False,1,0,0,0,0,0,0,"[{'championId': 80, 'pickTurn': 1}, {'champion...",3107,3853,3504,1052,3117,0,3364,2,4,14,2,1,1,1216,0,0,0,0,0,17577,12133,3317,2126,0,10598,7453,1146,1999,9732,5,8164,604,287,59,30,19826,7418,11923,484,7163,6610,0,6,4,4.0,0.0,168,12,3,0,27.0,4.0,False,False,False,False,False,False,8214,768,635,0,8226,250,407,0,8234,12213,0,0,8237,484,0,0,8345,3,0,0,8347,0,0,0,8200,8300,5008,5008,5003,"{'10-20': 0.4, '0-10': 0, '20-30': 0.2}","{'10-20': 305.9, '0-10': 294.5, '20-30': 332.7}","{'10-20': 240, '0-10': 213.5, '20-30': 199.6}","{'10-20': 489.3, '0-10': 380.6, '20-30': 1071.3}",NONE,JUNGLE,"{'10-20': -4.6, '0-10': -4.55, '20-30': -2.9}","{'10-20': -252.59999999999997, '0-10': -153.70...","{'10-20': -133.44999999999993, '0-10': -69.349...",GCHrpTgHUKbSY0ZorVd8SsTGQYlGDZSie1mz-XH5-ua9jQ,s3PUgsZ49uZbXUaJffMFJNRktGmFDxv1IgBRFKKRVfwZXng,NA1,GCHrpTgHUKbSY0ZorVd8SsTGQYlGDZSie1mz-XH5-ua9jQ,/v1/stats/player_history/NA1/41939982,3178


# Section 3: Analysis
Now that we've got our data wrangled into a master frame, we can get to our analysis. </p>

Let's identifiy some of our goals:
- How much damage does contribute summoner contribute in victories vs defeats?
- Kill and Death locations in wins vs losses

#### Analysis 3.1 Damage Share

In [24]:
def WinLossDamageShare(name, aggregated_df, lane=None):
    summ_filter = master['summonerName'].str.lower() == lookup_name.lower()             #str.lower() for dataframes
    lst = [(row.gameId, row.teamId) for key, row in master[summ_filter].iterrows()]
    df_lst = [row for key, row in master.iterrows() if ((row.gameId, row.teamId) in lst)]
    dfallies = pd.DataFrame(df_lst)

    #print messages
    print(f"{name}'s Damage Analsysis: Raw Numbers")
    print(aggregated_df[summ_filter].groupby('win').agg({'totalDamageDealtToChampions': [np.mean, np.size]}))
    print('\n\n %Team Damage Share in Wins and Losses')
    print( aggregated_df[summ_filter].groupby('win').sum().totalDamageDealtToChampions /
    dfallies.groupby('win').sum().totalDamageDealtToChampions)

    return

The below cell displays the average total damage done by the summoner (using the specified queue and champion filters from earlier) by wins and losses. 

<p> <b> Win: False</b> indicates stats in a loss. </p>
<p> <b> Win: True</b> indicates stats in a win </p>

In [25]:
WinLossDamageShare('JiggleMyJag', master)

JiggleMyJag's Damage Analsysis: Raw Numbers
      totalDamageDealtToChampions     
                             mean size
win                                   
False                22321.434783   46
True                 23016.461538   52


 %Team Damage Share in Wins and Losses
win
False    0.222281
True     0.231129
Name: totalDamageDealtToChampions, dtype: float64
